<a href="https://colab.research.google.com/github/ralik45/Padi_Care/blob/main/Rice_Leaf_Diseases_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'rice-diseases:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4111411%2F7227621%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240810%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240810T091658Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D387dc1ed02b36c728408a8dd712ad7f2cc41e90a0cf7c887b5dab711a9572b776563c3038244208a79e850aa8547963a0bbaf1fec35867d87c3206265e8fc6342e9a184f4c7402b6cae81e98f8755544703ef03a1edf5bf128ed89fa2d28a1d2c32e9504e5f7a175339be1260ce125e35eb5f65d0483ff1138b37720149f0dd6f9093258cc0a69f877e711d5829fb9e37402d61b9b13b13f477dd36aa0424b17ff35723d63de22d8f97d3a9121f0e80f5dd0ba5d39acc31be426cd31c5ca647f4b12cbe67d97fd4949e25d897ad61c85355e0b522adcba67b6ff50b88b452fec64c5d5257d32bedf3c10eb494175e1e947dcfb1f10615503e93220e0fc97a484,rl-diseases:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5025430%2F8474351%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240810%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240810T091658Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D07f6c374da14509c7abf82d2ef199a7547e5ea3a4df954553de49bb0e84272b24bdef7571cad20a2f3e31aa85c0693125358d07c5abf65ec5525024b82cacc0bd23b2aa6def955e70d2167d2b46979ec162ac7f8dfa909d008a6bf08f23764a94129b93c4941aa1c7876a9df5f85a53efccc90e08e4ed1e4c3e557df32f2afc2f252c446e66410f36aba921e94b5e1e256751ad54d6512749d555ff6e4de41b1ea5156282f5a7ae223f49e0aef09cf9cc76f3b381fa5b392a656b363293dc309cb73724740211de9def9098d91e3654abd006d073b1fb6031edca49f263a6d56671785fde9e6afbe35f49908ddc0fac8c758cc20b115cbe85b62e929840fed62'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Rice Leaf Diseases Detection

Rice Leaf Diseases Detection ini merupakan salah satu fitur dari project yang sedang kami buat dalam rangka menyelesaikan tugas capstone project

## Input

### Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, callbacks
from tensorflow.keras.layers import Flatten, Dense, Dropout

### Dataset

In [ ]:
train_data = '/kaggle/input/rl-diseases/RL_disease/RL_disease/train'
valid_data = '/kaggle/input/rl-diseases/RL_disease/RL_disease/valid'
testing_data = '/kaggle/input/rl-diseases/Test/Test'

In [ ]:
classes=[]
for file in os.listdir(train_data):
    classes+=[file]
print(classes)
print(len(classes))

In [ ]:
brownspot = [train_data + '/BrownSpot/' + img for img in os.listdir(train_data + '/BrownSpot')[:9]]
healthy = [train_data  + '/Healthy/' + img for img in os.listdir(train_data + '/Healthy')[:9]]
hispa = [train_data  + '/Hispa/' + img for img in os.listdir(train_data + '/Hispa')[:9]]
leafblast = [train_data  + '/LeafBlast/' + img for img in os.listdir(train_data + '/LeafBlast')[:9]]

#### Brown Spot

In [ ]:
from PIL import Image
plt.figure(figsize=(16,16))
for i,k  in enumerate(brownspot):
    image = Image.open(k)
    plt.subplot(3,3,i+1)
    plt.imshow(image)
    plt.title("Brown Spot")

#### Hispa

In [ ]:
plt.figure(figsize=(16,16))
for i,k  in enumerate(hispa):
    image = Image.open(k)
    plt.subplot(3,3,i+1)
    plt.imshow(image)
    plt.title("Hispa")

#### Leaf Blast

In [ ]:
plt.figure(figsize=(16,16))
for i,k  in enumerate(leafblast):
    image = Image.open(k)
    plt.subplot(3,3,i+1)
    plt.imshow(image)
    plt.title("Leaf Blast")

#### Healthy

In [ ]:
plt.figure(figsize=(16,16))
for i,k  in enumerate(healthy):
    image = Image.open(k)
    plt.subplot(3,3,i+1)
    plt.imshow(image)
    plt.title("Healthy")

In [ ]:
dataGenerator = ImageDataGenerator(rescale= 1. / 255)
train_data = dataGenerator.flow_from_directory(
    train_data,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)
val_data = dataGenerator.flow_from_directory(
    valid_data,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

In [ ]:
# Access one batch from the training data
for images, labels in train_data:
    print("Shape of training images:", images.shape)
    break  # We break the loop as we only need to inspect one batch

# Access one batch from the validation data
for images, labels in val_data:
    print("Shape of validation images:", images.shape)
    break  # Break after getting one batch

## Model

### List Model

#### Resnet

In [ ]:
resnet_model = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
resnet_model.trainable = False

#### MobileNet

In [ ]:
mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
mobilenet_model.trainable = False

#### Inception

In [ ]:
inception_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
inception_model.trainable = False

#### Vgg

In [ ]:
vgg_model = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
vgg_model.trainable = False

### Train Model

In [ ]:
model = Sequential()

model.add(resnet_model)
model.add(Flatten())
model.add(Dense(4, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
#early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10) #callbacks=[early_stopping, reduce_lr])
#print(f"Training stopped at epoch: {early_stopping.stopped_epoch}")

## Save Model

In [ ]:
model.save('./model.h5')
model.save('./model')
model.save('./model.tflite')

## Evaluate

### Loss & Accuracy

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2)
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history.history))

### Classification Report

In [ ]:
dataGenerator = ImageDataGenerator(rescale= 1. / 255)
test_data = dataGenerator.flow_from_directory(
    testing_data,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

In [ ]:
class_dict = test_data.class_indices
li = list(class_dict.keys())
print(li)

#### Test Data

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(test_data)
y_pred = np.argmax(y_pred, axis=1)
print('Classification Report')
target_names =li
print(classification_report(test_data.classes, y_pred, target_names=target_names))

#### Val Data

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(val_data)
y_pred = np.argmax(y_pred, axis=1)
print('Classification Report')
target_names =li
print(classification_report(val_data.classes, y_pred, target_names=target_names))

## Confusion Matrix

#### Test Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming your class names are as follows
class_names = ['BrownSpot', 'Healthy', 'Hispa', 'LeafBlast']

cm = confusion_matrix(test_data.classes, y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


#### Val Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Assuming your class names are as follows
class_names = ['BrownSpot', 'Healthy', 'Hispa', 'LeafBlast']

cm = confusion_matrix(val_data.classes, y_pred)

# Plotting the confusion matrix
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Test Random

In [ ]:
from PIL import Image
from keras.models import load_model
# Load the trained model
model = model

# Define the image size and preprocessing function
img_size = (224, 224)
def preprocess_image(img_path):
    img = Image.open(img_path).convert('RGB')
    img = img.resize(img_size)
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Set the path to the image folder
test_dir = '/kaggle/input/rl-diseases/random/random'
img_folder = test_dir
class_names = list(train_data.class_indices.keys())
# Loop through all the images in the folder and make predictions
for filename in os.listdir(img_folder):
    img_path = os.path.join(img_folder, filename)
    img = preprocess_image(img_path)
    pred = model.predict(img)
    class_idx = np.argmax(pred)
    class_name = class_names[class_idx]
    print(f'Image: {filename}, Class: {class_name}')